# Advanced VQE usage

In previous tutorials, we have seen how to set up a basic [`VQE`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) algorithm, how to monitor its convergence through the `callback` option, and how to leverage the [Aer Estimator]() in `VQE` to perform noisy simulations. Now, we will see how to provide more advanced configuration parameters to explore the full range of [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) capabilities. In particular, this tutorial will cover the use of custom `initial point`s and `gradient`s.

It will also cover advanced simulator use such as using Aer with the Matrix Product State method.

In [1]:
from qiskit import Aer
from qiskit.opflow import X, Z, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal

Here, we will use the same operator as in other VQE algorithms tutorials.

In [2]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

print(f"Number of qubits: {H2_op.num_qubits}")

## Initial point

The `initial_point` parameter allows the optimization to begin at the given point, defined by a list of parameter values that will configure the ansatz. By default, the initial point is `None`, which means that VQE will choose one based on the initial state of the ansatz. If the ansatz does not provide a preferred state, a random initial point that fits within the ansatz bounds will be chosen. If an initial point is supplied by the user, it will take priority and be used - note though it must match in length to the number of parameters in the ansatz circuit.

One might wonder... *Why set yourself an initial point?* Well, this option can come in handy if you have a guess for a reasonable starting point for the problem or perhaps know information from a prior experiment.

To demonstrate this feature, let's first simply repeat the first working example from the [algorithms introduction](01_algorithms_introduction.ipynb#A-complete-working-example) tutorial to get a solution's optimal point with the reference Estimator (statevector simulation).

In [3]:
from qiskit.primitives import Estimator
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithns.minimum_eigensolvers import VQE

ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1000)

estimator = Estimator()
vqe = VQE(estimator, ansatz, slsqp)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)
optimizer_evals = result.optimizer_evals

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 65,
    'eigenstate': array([ 9.54859564e-05+0.j, -9.93766269e-01+0.j,  1.11483597e-01+0.j,
        1.76972625e-05+0.j]),
    'eigenvalue': -1.8572750175780233,
    'optimal_parameters': {   ParameterVectorElement(θ[3]): 6.092947703759512,
                              ParameterVectorElement(θ[2]): 0.5470754201428533,
                              ParameterVectorElement(θ[1]): 4.426962028746158,
                              ParameterVectorElement(θ[4]): -2.5983257006415026,
                              ParameterVectorElement(θ[5]): 1.5683259438037993,
                              ParameterVectorElement(θ[6]): -4.717618238771348,
                              ParameterVectorElement(θ[7]): 0.3602072544539939,
                              ParameterVectorElement(θ[0]): 4.2965202693703635},
    'optimal_point': array([ 4.29652027,  4.42696203,  0.54707542,  6.0929477 , -2.5983257 ,
        1.56832594, -4.71761824,  0.36020

Now, we can take the `optimal_point` from the above result and use it as the `initial_point` for a follow-up computation.

**Note:** `initial_point` is now a keyword-only argument of the `VQE` class (i.e, it must be set following the `keyword=value` syntax).

In [4]:
initial_pt = result.optimal_point

estimator = Estimator()
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1000)

vqe = VQE(estimator, ansatz, slsqp, initial_point=initial_pt)
result1 = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result1)

optimizer_evals1 = result1.optimizer_evals
print()
print(
    f"optimizer_evals is {optimizer_evals1} with initial point versus {optimizer_evals} without it."
)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 9,
    'eigenstate': array([ 9.54859564e-05+0.j, -9.93766269e-01+0.j,  1.11483597e-01+0.j,
        1.76972625e-05+0.j]),
    'eigenvalue': -1.8572750175780233,
    'optimal_parameters': {   ParameterVectorElement(θ[3]): 6.092947703759512,
                              ParameterVectorElement(θ[6]): -4.717618238771348,
                              ParameterVectorElement(θ[5]): 1.5683259438037993,
                              ParameterVectorElement(θ[1]): 4.426962028746158,
                              ParameterVectorElement(θ[2]): 0.5470754201428533,
                              ParameterVectorElement(θ[7]): 0.3602072544539939,
                              ParameterVectorElement(θ[4]): -2.5983257006415026,
                              ParameterVectorElement(θ[0]): 4.2965202693703635},
    'optimal_point': array([ 4.29652027,  4.42696203,  0.54707542,  6.0929477 , -2.5983257 ,
        1.56832594, -4.71761824,  0.360207

By looking at the `optimizer_time` we can notice how the initial point helped the algorithm reach a solution much faster. `optimizer_evals` shows how the number of evaluations dropped from 65 to 9.

This can be particularly useful in cases where we have two closely related problems, and the solution to one problem can be used to guess the other's. A good example might be plotting dissociation profiles in chemistry, where we change the inter-atomic distances of a molecule and compute its minimum eigenvalue for each distance. When the distance changes are small, we expect the solution to still be close to the prior one. Thus, a popular technique is to simply use the optimal point from one solution as the starting point for the next step. There also exist more complex techniques, where we can apply extrapolation to compute an initial position based on prior solution(s) rather than directly use the prior solution.

## Gradient

If the provided optimizer uses a gradient-based technique, the default gradient method will be finite differences. However, the `VQE` class includes an option to pass custom gradients via its `gradient` parameter.

As the use of a user supplied `gradient` was shown in the [Monitoring VQE Convergence](02_vqe_convergence.ipynb#Using-Gradient-framework) tutorial I will simply refer you there.

## Quantum Instance and advanced simulation

While you may be familiar with passing a QuantumInstancen created from a `aer_simulator_statevector` a `aer_simulator` or   real device backend, it is possible to use the advanced simulation modes of Aer too when applicable. For instance we can easily use the Aer [Matrix Product State](../simulators/7_matrix_product_state_method.ipynb) method, that has the potential to scale to larger numbers of qubits.

In [10]:
algorithm_globals.random_seed = seed

from qiskit.providers.aer import QasmSimulator

quantum_instance = QuantumInstance(
    QasmSimulator(method="matrix_product_state"), shots=1
)

ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi, include_custom=True)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 65,
    'eigenstate': {'01': 0.9921567416492215, '10': 0.125},
    'eigenvalue': -1.8572750175807682,
    'optimal_parameters': {   ParameterVectorElement(θ[2]): 0.5470752986013949,
                              ParameterVectorElement(θ[3]): 6.092947713510392,
                              ParameterVectorElement(θ[5]): 1.5683258132970883,
                              ParameterVectorElement(θ[7]): 0.3602071559531031,
                              ParameterVectorElement(θ[4]): -2.598325866938012,
                              ParameterVectorElement(θ[1]): 4.426962159645716,
                              ParameterVectorElement(θ[6]): -4.717618259450455,
                              ParameterVectorElement(θ[0]): 4.296520300933687},
    'optimal_point': array([ 4.2965203 ,  4.42696216,  0.5470753 ,  6.09294771, -2.59832587,
        1.56832581, -4.71761826,  0.36020716]),
    'optimal_value': -1.8572750175807682,
    'optimiz

In [11]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright